# Tic Tac Toe
- 3^9 = 19683 possible states
- many of them are forbidden by the game logic (e.g. all X or all O)

## 1. Step: Generate all possible states

In [1]:
v = ["-", "O", "X"]
boardStateArray = Array{Array{Any, 1}, 1}(undef, 3^9)

for i in 1:3^9
    currentBoardStateVector = Array{Any, 1}(undef, 9)
    c = i
    for j in 1:9
        currentBoardStateVector[j] = v[floor(Int, (c%3+1))]
        c /= 3
    end

    boardStateArray[i] = currentBoardStateVector
end

println("Generated ", length(boardStateArray), " possible board states.")
println("E.g. the board state nr ", 3^9-5, " is: ", boardStateArray[3^9-5])

Generated 19683 possible board states.
E.g. the board state nr 19678 is: 

Any

["O", "O", "X", "X", "X", "X", "X", "X", "X"]


## 2. Step: Remove all impossible states

First remove all states which would mean one player overjumped the other one. Which means there are two more X or O on the field compared to the opponent.

In [10]:
count = 0

for boardState in boardStateArray
    countX = 0
    countO = 0

    for i in 1:9
        if boardState[i] == "X"
            countX += 1
        elseif boardState[i] == "O"
            countO += 1
        end
    end

    if (countX > countO+1) || (countO > countX+1)
        # TODO: Mark invalid ones with a second array.
        count += 1
    end
end

println(count, " of ", length(boardStateArray), " are invalid (", (count/length(boardStateArray))*100, "%)")
# TODO: Print() one example to proof the logic: println("One invalid example: ", )
# TODO: delete all invalid entries from boardStateArray

10730 of 19683 are invalid (54.514047655337095%)
